In [236]:
import requests
import json
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [237]:
df = pd.read_csv('./data/최종_최종__월별기상관측.csv', encoding='cp949')
df = df.drop(['Unnamed: 0'], axis=1)

In [238]:
region = df['행정구역'].unique()
region

array(['강원도', '경기도', '인천광역시', '서울특별시', '경상북도', '충청북도', '충청남도', '대전광역시',
       '전라남도', '대구광역시', '전라북도', '울산광역시', '경상남도', '광주광역시', '부산광역시', '제주도',
       '세종특별자치시'], dtype=object)

In [239]:
# g = df[(df['행정구역'] == '강원도')]

In [240]:
# g

In [241]:
# t_date = g['일시'].unique()

In [242]:
# t_date

In [243]:
#t = g[(g['일시'] == '2000-01')]

In [244]:
#t

In [245]:
#t.describe()['평균기온(°C)']

In [246]:
#t.describe()['평균기온(°C)']['50%']

In [247]:
def calc_median(df:pd.DataFrame, colname:str):
    # df(데이터 프레임) colname의 중앙값 산출하는 함수
    return df.describe()[colname]['50%']

In [248]:
reg_df = pd.DataFrame()

for reg in region:
    print(f'행정구역 : {reg}')
    temp_mean_list, temp_max_list, temp_min_list, rain_sum_list, sun_sum_list = [],[],[],[],[] # 중앙값 데이터 쌓을 빈 리스트 생성 (초기화)
    
    d = df[(df['행정구역'] == reg)] # 행정구역의 df 생성.
    date_list = d['일시'].unique() # 일시 리스트 생성.
    
    for date in date_list:
        r = d[(d['일시'] == date)] # 예 : 행정구역이 강원도 이면서 일시가 2000-01 인 데이터 프레임
        
        # 컬럼별 중앙값 구하기.
        temp_mean = calc_median(r, '평균기온(°C)')
        temp_max = calc_median(r, '최고기온(°C)')
        temp_min = calc_median(r, '최저기온(°C)')
        rain_sum = calc_median(r, '월합강수량(00~24h만)(mm)')
        sun_sum = calc_median(r, '합계 일사량(MJ/m2)')
        
        # 구한 중앙값들을 각 리스트에 쌓기.
        temp_mean_list.append(temp_mean)
        temp_max_list.append(temp_max)
        temp_min_list.append(temp_min)
        rain_sum_list.append(rain_sum)
        sun_sum_list.append(sun_sum)
    # 임의의 데이터 프레임으로 만들기. 
    imsi_df = pd.DataFrame({'행정구역':reg, '일시':date_list,'평균기온(°C)': temp_mean_list,
                           '최고기온(°C)': temp_max_list, '최저기온(°C)': temp_min_list,
                           '월합강수량(00~24h만)(mm)':rain_sum_list,'합계 일사량(MJ/m2)':sun_sum_list})
    # 위에서 생성한 빈 reg_df와 임의의 데이터 프레임 합치기
    reg_df = pd.concat([reg_df, imsi_df], axis=0)
reg_df.reset_index(inplace=True, drop=True)
        

행정구역 : 강원도
행정구역 : 경기도
행정구역 : 인천광역시
행정구역 : 서울특별시
행정구역 : 경상북도
행정구역 : 충청북도
행정구역 : 충청남도
행정구역 : 대전광역시
행정구역 : 전라남도
행정구역 : 대구광역시
행정구역 : 전라북도
행정구역 : 울산광역시
행정구역 : 경상남도
행정구역 : 광주광역시
행정구역 : 부산광역시
행정구역 : 제주도
행정구역 : 세종특별자치시


In [249]:
df = reg_df

In [250]:
# 봄배추 3월~5월
# 고랭지배추 5월~9월
# 가을배추 7월~11월
# 겨울배추 8월~내년2월
#평균기온, 최저기온, 최고기온, 월합강수량, 합계일사량 

In [251]:
# 일시 뽑을 방법 확인
df['일시'][0][-2:]

'01'

In [252]:
# 월만 추가
# month = df['일시'].apply(lambda x:x.split('-')[-1])
# df.insert(3, '월', month)
df['월'] = df['일시'].apply(lambda x:x.split('-')[-1])
df.head()

,행정구역,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),월
0,강원도,2000-01,-2.9,8.5,-16.0,48.1,152.990,01
1,강원도,2000-02,-3.3,7.7,-15.8,5.7,329.070,02
2,강원도,2000-03,4.4,19.3,-8.6,14.1,448.650,03
3,강원도,2000-04,10.4,23.7,-3.1,37.1,486.355,04
4,강원도,2000-05,15.9,31.6,3.9,75.2,520.280,05


In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4488 entries, 0 to 4487
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   행정구역                4488 non-null   object 
 1   일시                  4488 non-null   object 
 2   평균기온(°C)            4488 non-null   float64
 3   최고기온(°C)            4488 non-null   float64
 4   최저기온(°C)            4488 non-null   float64
 5   월합강수량(00~24h만)(mm)  4488 non-null   float64
 6   합계 일사량(MJ/m2)       4488 non-null   float64
 7   월                   4488 non-null   object 
dtypes: float64(5), object(3)
memory usage: 280.6+ KB


In [254]:
#겨울배추 추가
winter_df = df.loc[(df['월'] == '01')|(df['월'] == '02')|(df['월'] == '08')|(df['월'] == '09')|(df['월'] == '10')|(df['월'] == '11')|(df['월'] == '12')]

In [255]:
#봄배추, 고랭지배추, 가을배추 추가
spring_df =  df.loc[(df['월'] == '03')|(df['월'] == '04')|(df['월'] == '05')]
gorange_df = df.loc[(df['월'] == '05')|(df['월'] == '06')|(df['월'] == '07')|(df['월'] == '08')|(df['월'] == '9')]
fall_df = df.loc[(df['월'] == '07')|(df['월'] == '08')|(df['월'] == '09')|(df['월'] == '10')|(df['월'] == '11')]

In [256]:
spring_df

,행정구역,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),월
2,강원도,2000-03,4.4,19.3,-8.6,14.1,448.650,03
3,강원도,2000-04,10.4,23.7,-3.1,37.1,486.355,04
4,강원도,2000-05,15.9,31.6,3.9,75.2,520.280,05
14,강원도,2001-03,3.3,21.6,-8.5,20.6,445.030,03
15,강원도,2001-04,12.0,27.3,-4.1,16.5,539.605,04
...,...,...,...,...,...,...,...,...
4467,세종특별자치시,2020-04,10.8,25.9,-0.8,17.6,651.275,04
4468,세종특별자치시,2020-05,18.2,29.1,7.5,91.6,643.645,05
4478,세종특별자치시,2021-03,8.8,23.3,-2.3,99.8,465.040,03
4479,세종특별자치시,2021-04,13.9,29.8,0.7,56.0,585.900,04


In [257]:
spring_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1122 entries, 2 to 4480
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   행정구역                1122 non-null   object 
 1   일시                  1122 non-null   object 
 2   평균기온(°C)            1122 non-null   float64
 3   최고기온(°C)            1122 non-null   float64
 4   최저기온(°C)            1122 non-null   float64
 5   월합강수량(00~24h만)(mm)  1122 non-null   float64
 6   합계 일사량(MJ/m2)       1122 non-null   float64
 7   월                   1122 non-null   object 
dtypes: float64(5), object(3)
memory usage: 78.9+ KB


In [258]:
# 배추품종에 따라 값을 부여함

winter_df['배추품종'] = '겨울배추'
spring_df['배추품종'] = '봄배추'
gorange_df['배추품종'] = '고랭지배추'
fall_df['배추품종'] = '겨울배추'

In [259]:
spring_df

,행정구역,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),월,배추품종
2,강원도,2000-03,4.4,19.3,-8.6,14.1,448.650,03,봄배추
3,강원도,2000-04,10.4,23.7,-3.1,37.1,486.355,04,봄배추
4,강원도,2000-05,15.9,31.6,3.9,75.2,520.280,05,봄배추
14,강원도,2001-03,3.3,21.6,-8.5,20.6,445.030,03,봄배추
15,강원도,2001-04,12.0,27.3,-4.1,16.5,539.605,04,봄배추
...,...,...,...,...,...,...,...,...,...
4467,세종특별자치시,2020-04,10.8,25.9,-0.8,17.6,651.275,04,봄배추
4468,세종특별자치시,2020-05,18.2,29.1,7.5,91.6,643.645,05,봄배추
4478,세종특별자치시,2021-03,8.8,23.3,-2.3,99.8,465.040,03,봄배추
4479,세종특별자치시,2021-04,13.9,29.8,0.7,56.0,585.900,04,봄배추


In [260]:
# 생산량 데이터 불러와서 . . . 전처리 . . 
# pandas wide to long (검색해서 배추 생산량 데이터 ,, 데이터 프레임 형태로 만들기
# 16일은 데이터 프레임 형태 이후 합치고 전처리

In [261]:
import pandas as pd
#데이터 프레임 만들기


In [262]:
# 생산량 데이터
produce_df = pd.read_csv('./배추_생산량.csv')

# 배추별 특징(겨울배추는 2014년 부터 있음)
# 노지봄배추 = 노지봄배추 + 일반봄배추
# 노지배추 = 일반봄배추 + 고랭지배추 + 노지가을배추 + 노지겨울배추
# 배추 = 노지봄배추 + 일반봄배추  + 고랭지배추 + 노지가을배추 + 노지겨울배추 + 시설배추

In [263]:
region2 = produce_df.iloc[1:, :1]

year = [y.split('.')[0] for y in produce_df.iloc[:, 1:].columns] #년도에서 .을 기준으로 뒤에 제거 후 저장(년도만)
handle_df = produce_df.iloc[:, 1:] #시도별 열을 제외한 데이터 

In [264]:
#a = [y.split('.')[0] for y in produce_df.iloc[:, 1:].columns]
#a

In [265]:
# produce_df.iloc[:, 1:] 

In [266]:
handle_df.columns = [handle_df[col][0] for col in handle_df.columns]
handle_df = handle_df.iloc[1:]

In [267]:
#handle_df

In [268]:
handle_df.insert(0, '행정구역', region2['시도별'].tolist())

In [269]:
produce_df2 = pd.melt(handle_df, id_vars=['행정구역'])
produce_df2

,행정구역,variable,value
0,계,엽채류:면적 (ha),72925
1,서울특별시,엽채류:면적 (ha),723
2,부산광역시,엽채류:면적 (ha),1392
3,대구광역시,엽채류:면적 (ha),1167
4,인천광역시,엽채류:면적 (ha),1154
...,...,...,...
9139,전라북도,생산량 (톤),14189
9140,전라남도,생산량 (톤),30477
9141,경상북도,생산량 (톤),5960
9142,경상남도,생산량 (톤),2419


In [270]:
handle_df.columns.unique()

Index(['행정구역', '엽채류:면적 (ha)', '생산량 (톤)', '배추:면적 (ha)', '노지배추:면적 (ha)',
       '10a당 생산량 (kg)', '노지봄배추:면적 (ha)', '일반봄배추:면적 (ha)', '고랭지배추:면적 (ha)',
       '노지가을배추:면적 (ha)', '시설배추:면적 (ha)', '노지겨울배추:면적 (ha)'],
      dtype='object')

In [271]:
# produce_df.iloc[:, 1:]

In [272]:
# produce_df.iloc[1:, :1]

In [273]:
df_2000 = produce_df.iloc[1:, :23]
df_2001 = produce_df.iloc[1:, 23:45]
df_2002 = produce_df.iloc[1:, 45:67]
df_2003 = produce_df.iloc[1:, 67:89]
df_2004 = produce_df.iloc[1:, 89:111]
df_2005 = produce_df.iloc[1:, 111:133]
df_2006 = produce_df.iloc[1:, 133:155]
df_2007 = produce_df.iloc[1:, 155:177]
df_2008 = produce_df.iloc[1:, 177:199]
df_2009 = produce_df.iloc[1:, 199:221]
df_2010 = produce_df.iloc[1:, 221:243]
df_2011 = produce_df.iloc[1:, 243:265]
df_2012 = produce_df.iloc[1:, 265:287]
df_2013 = produce_df.iloc[1:, 287:309]
df_2014 = produce_df.iloc[1:, 309:334]
df_2015 = produce_df.iloc[1:, 334:359]
df_2016 = produce_df.iloc[1:, 359:384]
df_2017 = produce_df.iloc[1:, 384:409]
df_2018 = produce_df.iloc[1:, 409:434]
df_2019 = produce_df.iloc[1:, 434:459]
df_2020 = produce_df.iloc[1:, 459:484]
df_2021 = produce_df.iloc[1:, 484:509]

In [274]:
# df_2000 = produce_df.iloc[1:, 1:23]
# df_2000


In [275]:
# df_2000 = produce_df.iloc[1:, 1:23]
# df_2000.columns = ['엽채류:면적 (ha)', '생산량 (톤)','배추:면적 (ha)', '생산량 (톤)', '노지배추:면적 (ha)',
#        '10a당 생산량 (kg)', '생산량 (톤)', '노지봄배추:면적 (ha)', '일반봄배추:면적 (ha)', '고랭지배추:면적 (ha)',
#        '노지가을배추:면적 (ha)', '시설배추:면적 (ha)', '노지겨울배추:면적 (ha)']

In [276]:
# new_data = pd.concat([df_2000, df_2001], axis=0)

In [277]:
# new_data

In [278]:
# 다시불러오기
df2 = pd.read_csv('./data/배추_생산량.csv')

In [279]:
df2.head()

,시도별,2000,2000.1,2000.2,2000.3,2000.4,2000.5,2000.6,2000.7,2000.8,...,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11
0,품목별,일반봄배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),고랭지배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),노지가을배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),...,생산량 (톤),고랭지배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),노지가을배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),노지겨울배추:면적 (ha),10a당 생산량 (kg),생산량 (톤)
1,서울특별시,72,3695,2660,0,0,0,23,10152,2335,...,228,0,0,0,6,12793,771,0,0,0
2,부산광역시,403,4446,17917,0,0,0,324,10835,35105,...,259,0,0,0,36,11239,4017,0,0,0
3,대구광역시,177,4799,8494,0,0,0,139,10657,14813,...,1343,0,0,0,68,7248,4895,0,0,0
4,인천광역시,107,4316,4618,0,0,0,166,9907,16446,...,378,0,0,0,138,9664,13333,0,0,0


In [280]:
def new_columns(collist):
    new_colnames = []
    for i in range(len(collist[:])):
        if ':' in collist[:][i]:
            new_colnames.append(collist[:][i])
        else:
            if ':' in collist[:][i-1]:
#                 print(collist[:][i-1])
                new_colname = collist[:][i-1].split(':')[0] + ':' + collist[:][i]
                new_colnames.append(new_colname)
            elif ':' in collist[:][i-2]:
                new_colname = collist[:][i-2].split(':')[0] + ':' + collist[:][i]
                new_colnames.append(new_colname)
    return new_colnames

In [281]:
region2 = df2.iloc[1:, :1]['시도별'].tolist() #행정구역 명 
output_df = pd.DataFrame()
for y in range(2000, 2022):
    y_list = []
    for col in df2.columns:
        if col.split('.')[0] == str(y):
#             print(col.split('.')[0])
            y_list.append(col)
    print(f'{y} : {len(y_list)}')
    handle_df = df2[y_list]
    
    handle_df.columns = [handle_df[col][0] for col in handle_df.columns]
    handle_df.columns = new_columns(handle_df.columns) #칼럼명 매칭
    
    
    handle_df = handle_df.iloc[1:]
    handle_df.insert(0, '행정구역', region2)
    e_df2 = pd.melt(handle_df, id_vars=['행정구역']) #행정구역을 기준으로 wide 데이터 long 변환
    e_df2.insert(0, '일시', str(y)) #일시 추가
    output_df = pd.concat([output_df, e_df2], axis=0)
output_df.reset_index(inplace=True, drop=True)

2000 : 9
2001 : 9
2002 : 9
2003 : 9
2004 : 9
2005 : 9
2006 : 9
2007 : 9
2008 : 9
2009 : 9
2010 : 9
2011 : 9
2012 : 9
2013 : 9
2014 : 12
2015 : 12
2016 : 12
2017 : 12
2018 : 12
2019 : 12
2020 : 12
2021 : 12


In [282]:
tgt = ['일반봄배추', '고랭지배추', '노지가을배추', '노지겨울배추']
tgt_index = []
for i in range(len(output_df)):
    if output_df['variable'][i].split(':')[0] in tgt:
        print(output_df['variable'][i])
        tgt_index.append(i)

일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:면적 (ha)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:10a당 생산량 (kg)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
일반봄배추:생산량 (톤)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭지배추:면적 (ha)
고랭

In [283]:
print(len(tgt_index))

3774


In [284]:
output_df = output_df.loc[tgt_index]
output_df.reset_index(inplace=True, drop=True)

In [285]:
split_spcies = [v.split(':')[0] for v in output_df['variable']]
output_df.insert(3, '품종', split_spcies)
output_df['variable'] = [v.split(':')[-1] for v in output_df['variable']]  # :기준으로 분리

In [286]:
# 품종에 따른, 생산량 , 면적, 10a당 생산량 
# 면적 (ha) 생산량 (톤) 10a당 생산량 (kg)
def species_df(df, species):
    spec_df = df.loc[(df['품종'] == species)]
    spec_df.reset_index(inplace=True, drop=True)
    
    idx_l = [i for i in range(len(spec_df)) if spec_df['variable'][i] == '면적 (ha)']
    area_list = [spec_df['value'][i] for i in range(len(spec_df)) if spec_df['variable'][i] == '면적 (ha)']
    out_list = [spec_df['value'][i] for i in range(len(spec_df)) if spec_df['variable'][i] == '생산량 (톤)']
    out_10a_list = [spec_df['value'][i] for i in range(len(spec_df)) if spec_df['variable'][i] == '10a당 생산량 (kg)']
    
    result_df = spec_df.loc[idx_l]
    result_df = result_df[['일시', '행정구역', '품종']]
    
    result_df['면적'] = area_list
    result_df['생산량'] = out_list
    result_df['10a당_생산량'] = out_10a_list 
    
    result_df.reset_index(inplace=True, drop=True)
    return result_df

In [287]:
output_spring_df = species_df(output_df, '일반봄배추')
output_winter_df = species_df(output_df, '노지겨울배추')
output_grange_df = species_df(output_df, '고랭지배추')
output_fall_df = species_df(output_df, '노지가을배추')

In [288]:
output_spring_df

,일시,행정구역,품종,면적,생산량,10a당_생산량
0,2000,서울특별시,일반봄배추,72,2660,3695
1,2000,부산광역시,일반봄배추,403,17917,4446
2,2000,대구광역시,일반봄배추,177,8494,4799
3,2000,인천광역시,일반봄배추,107,4618,4316
4,2000,광주광역시,일반봄배추,178,9060,5090
...,...,...,...,...,...,...
369,2021,전라북도,일반봄배추,449,24901,5552
370,2021,전라남도,일반봄배추,1250,72993,5839
371,2021,경상북도,일반봄배추,1231,60477,4912
372,2021,경상남도,일반봄배추,155,7273,4704


In [289]:
output_winter_df

,일시,행정구역,품종,면적,생산량,10a당_생산량
0,2014,서울특별시,노지겨울배추,0,0,0
1,2014,부산광역시,노지겨울배추,0,0,0
2,2014,대구광역시,노지겨울배추,0,0,0
3,2014,인천광역시,노지겨울배추,0,0,0
4,2014,광주광역시,노지겨울배추,0,0,0
...,...,...,...,...,...,...
131,2021,전라북도,노지겨울배추,0,0,0
132,2021,전라남도,노지겨울배추,3236,224283,6931
133,2021,경상북도,노지겨울배추,0,0,0
134,2021,경상남도,노지겨울배추,0,0,0


In [290]:
output_grange_df

,일시,행정구역,품종,면적,생산량,10a당_생산량
0,2000,서울특별시,고랭지배추,0,0,0
1,2000,부산광역시,고랭지배추,0,0,0
2,2000,대구광역시,고랭지배추,0,0,0
3,2000,인천광역시,고랭지배추,0,0,0
4,2000,광주광역시,고랭지배추,0,0,0
...,...,...,...,...,...,...
369,2021,전라북도,고랭지배추,89,2868,3214
370,2021,전라남도,고랭지배추,0,0,0
371,2021,경상북도,고랭지배추,312,11997,3850
372,2021,경상남도,고랭지배추,8,380,4741


In [291]:
output_fall_df

,일시,행정구역,품종,면적,생산량,10a당_생산량
0,2000,서울특별시,노지가을배추,23,2335,10152
1,2000,부산광역시,노지가을배추,324,35105,10835
2,2000,대구광역시,노지가을배추,139,14813,10657
3,2000,인천광역시,노지가을배추,166,16446,9907
4,2000,광주광역시,노지가을배추,195,20352,10437
...,...,...,...,...,...,...
369,2021,전라북도,노지가을배추,1296,120590,9304
370,2021,전라남도,노지가을배추,3226,320451,9933
371,2021,경상북도,노지가을배추,1712,124232,7256
372,2021,경상남도,노지가을배추,477,43845,9190


# 기후 데이터 연별로 합치기

In [292]:
# winter_df['배추품종'] = '겨울배추'
# spring_df['배추품종'] = '봄배추'
# gorange_df['배추품종'] = '고랭지배추'
# fall_df['배추품종'] = '겨울배추'

In [1]:
winter_df

NameError: name 'winter_df' is not defined

In [294]:
# winter_df에서 년도 컬럼 삭제
# winter_df = winter_df.drop(['년도'], axis=1)

In [295]:
# winter_df 에서 일시 컬럼을 년도, 월, 일로 분리
winter_df['년도'] = winter_df['일시'].str.split('-').str[0]
winter_df

# spring_df 일시 컬럼을 년도, 월로 분리
spring_df['년도'] = spring_df['일시'].str.split('-').str[0]
spring_df


# gorange_df 일시 컬럼을 년도, 월로 분리
gorange_df['년도'] = gorange_df['일시'].str.split('-').str[0]
gorange_df

# fall_df 일시 컬럼을 년도, 월로 분리
fall_df['년도'] = fall_df['일시'].str.split('-').str[0]
fall_df

,행정구역,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),월,배추품종,년도
6,강원도,2000-07,24.9,34.3,16.7,153.4,512.230,07,겨울배추,2000
7,강원도,2000-08,24.1,33.7,17.4,348.0,412.735,08,겨울배추,2000
8,강원도,2000-09,18.5,31.1,8.5,256.9,347.645,09,겨울배추,2000
9,강원도,2000-10,12.1,26.1,-1.6,16.9,311.070,10,겨울배추,2000
10,강원도,2000-11,4.1,20.3,-8.4,30.2,231.670,11,겨울배추,2000
...,...,...,...,...,...,...,...,...,...,...
4482,세종특별자치시,2021-07,26.9,34.8,19.9,180.7,639.565,07,겨울배추,2021
4483,세종특별자치시,2021-08,25.1,34.5,18.6,232.1,445.145,08,겨울배추,2021
4484,세종특별자치시,2021-09,21.8,29.4,14.4,184.6,426.565,09,겨울배추,2021
4485,세종특별자치시,2021-10,15.0,29.8,0.7,36.4,407.200,10,겨울배추,2021


In [296]:
# gorange_df

In [297]:
# fall_df['일시'].str.split('-').str[1]

In [298]:
# winter_df 행정구역별로 중앙값 구하기
winter_df_median = winter_df.groupby(['행정구역', '년도']).median()

# spring_df 행정구역별로 중앙값 구하기
spring_df_median = spring_df.groupby(['행정구역', '년도']).median()

# gorange_df 행정구역별로 중앙값 구하기
gorange_df_median = gorange_df.groupby(['행정구역', '년도']).median()

# fall_df 행정구역별로 중앙값 구하기
fall_df_median = fall_df.groupby(['행정구역', '년도']).median()

In [299]:
# winter_df_median 배추품종 컬럼 삭제
# winter_df_median = winter_df_median.drop(['배추품종'], axis=1)


In [300]:
# winter_df_median 인덱싱 0으로 넣고 행적구역과 년도 컬럼으로 설정
winter_df_median = winter_df_median.reset_index(level=['행정구역', '년도'])
spring_df_median = spring_df_median.reset_index(level=['행정구역', '년도'])
gorange_df_median = gorange_df_median.reset_index(level=['행정구역', '년도'])
fall_df_median = fall_df_median.reset_index(level=['행정구역', '년도'])

In [301]:
fall_df_median

,행정구역,년도,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,강원도,2000,18.5,31.1,8.5,153.4,347.645
1,강원도,2001,19.0,30.7,4.1,67.3,411.455
2,강원도,2002,18.0,31.1,9.3,75.0,365.845
3,강원도,2003,19.0,28.7,7.6,277.7,388.365
4,강원도,2004,19.6,30.0,11.0,193.6,403.205
...,...,...,...,...,...,...,...
369,충청북도,2017,18.9,28.9,5.5,78.3,428.255
370,충청북도,2018,19.2,29.2,6.6,161.5,440.480
371,충청북도,2019,20.9,31.5,10.1,102.6,403.115
372,충청북도,2020,18.7,28.6,8.7,160.6,439.525


In [302]:
# winter_df_mean 년도 컬럼명을 일시로 변경
winter_df_median = winter_df_median.rename(columns={'년도': '일시'})
spring_df_median = spring_df_median.rename(columns={'년도':'일시'})
gorange_df_median = gorange_df_median.rename(columns={'년도':'일시'})
fall_df_median = fall_df_median.rename(columns={'년도':'일시'})

In [303]:
# winter_df_mean 과 output_spring_df 병합
winter_df_median = pd.merge(winter_df_median, output_winter_df, on=['행정구역', '일시'])
spring_df_median = pd.merge(spring_df_median, output_winter_df, on=['행정구역', '일시'])
gorange_df_median = pd.merge(gorange_df_median, output_winter_df, on=['행정구역', '일시'])
fall_df_median = pd.merge(fall_df_median, output_winter_df, on=['행정구역', '일시'])

In [304]:
# 10a당_생산량 컬럼 삭제
winter_df_median = winter_df_median.drop(['10a당_생산량'], axis=1)
spring_df_median = spring_df_median.drop(['10a당_생산량'], axis=1)
gorange_df_median = gorange_df_median.drop(['10a당_생산량'], axis=1)
fall_df_median = fall_df_median.drop(['10a당_생산량'], axis=1)

In [312]:
spring_df_median

,행정구역,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),품종_x,면적_x,생산량_x,품종_y,면적_y,생산량_y,10a당_생산량
0,강원도,2014,13.10,26.60,-0.7,34.8,489.360,노지겨울배추,0,0,일반봄배추,452,18627,4121
1,강원도,2015,11.40,29.30,0.6,23.7,504.380,노지겨울배추,0,0,일반봄배추,652,25506,3912
2,강원도,2016,13.50,28.00,1.1,60.0,503.490,노지겨울배추,0,0,일반봄배추,600,28872,4812
3,강원도,2017,13.35,28.60,-0.3,25.2,572.235,노지겨울배추,0,0,일반봄배추,842,47271,5616
4,강원도,2018,12.20,29.35,-1.5,135.6,526.820,노지겨울배추,0,0,일반봄배추,646,31063,4805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,충청북도,2017,13.40,28.30,0.3,22.5,559.045,노지겨울배추,0,0,일반봄배추,1015,43757,4311
132,충청북도,2018,12.70,30.00,-1.5,103.9,555.240,노지겨울배추,0,0,일반봄배추,711,34482,4850
133,충청북도,2019,11.40,28.60,-4.7,29.0,509.800,노지겨울배추,0,0,일반봄배추,719,32762,4555
134,충청북도,2020,10.20,25.60,-3.1,28.9,643.645,노지겨울배추,0,0,일반봄배추,595,27072,4550


In [306]:
# 병합 winter_df_mean 겨울배추 , spiring_df_mean 봄배추, gorange_df_mean 여름배추, fall_df_mean 가을배추
spring_df_median = pd.merge(spring_df_median, output_spring_df, on=['행정구역', '일시'])
gorange_df_median = pd.merge(gorange_df_median, output_grange_df, on=['행정구역', '일시'])
fall_df_median = pd.merge(fall_df_median, output_fall_df, on=['행정구역', '일시'])

In [307]:
# 지역별로 나누기 
# busan_df_winter = winter_df_median[(winter_df_median['행정구역'] == '부산광역시')]
# busan_df_winter

In [308]:
# # winter_df_median, spring_df_median, gorange_df_median, fall_df_median 병합
# df_sum_median = pd.concat([winter_df_median, spring_df_median, gorange_df_median, fall_df_median], axis=0)
# df_sum_median

In [309]:
# NaN 값이 너무 많음.. 
# df_sum_median.isna().sum()

In [311]:
# x = spring_df_median[['평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)', '면적']]
# y = spring_df_median[['생산량']]

KeyError: "['면적'] not in index"

KeyError: "['최저기온(°C)월합강수량(00~24h만)(mm)', '면적'] not in index"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)

In [ ]:
spring_df_median